# Evaluating Fairness of AI Models in Radiology

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/Sulam-Group/AI-Deep-Learning-Lab-2023/blob/bbjt-nb-fairness_interpretability/sessions/ai-fairness/fairness.ipynb)